# Encontrando una política para jugar el 21

Esta libreta utiliza un algoritmo de programación dinámica para encontrar una política que permita jugar al 21.

Las reglas para este 21 son especiales:
- El mazo es infinito. De esta manera las 13 cartas siempre tienen las mismas probabilidades de aparecer
- El jugador realiza dos acciones antes de que el repartidor juegue
- Quien llegue a 4 cartas sin pasarse automáticamente gana

Con las reglas explicadas, definiremos una estructura para procesos de decisión markoviana.

In [1]:
struct MDP
    states
    actions
    ρ
    reward
    final_s
end

## Función para hallar la política

Enseguida viene una implementación de un algoritmo iterativo para encontrar una buena política.

In [10]:
"""
    iter_value(mdp::MDP, γ::Float64)

Iteratively computes the value function of a Markov Decision Process using
discount rate γ and then returns the optimal policy π associated with it.
"""
function iter_value(mdp::MDP, γ::Float64)
    v = Dict(s => 0.0 for s in mdp.states)
    v_p = Dict(s => 0.0 for s in mdp.states)
    
    has_converged = false
    
    while !has_converged
        for s in keys(v)
            v_p[s] = maximum([sum([mdp.ρ(s, a, n_s) * (mdp.reward(s, a, n_s) + γ * v[n_s])
                                        for n_s in mdp.states])
                                    for a in mdp.actions])

            has_converged = true
                            
            for s in keys(v)
                if v_p[s] > v[s]
                    v[s] = v_p[s]
                    has_converged = false
                end
            end
            
            if has_converged
                break
            end
        end
    end
    
    pol_π = Dict(s => "" for s in mdp.states)
    
    for s in keys(v)
        actions_value = Dict(a => sum([mdp.ρ(s, a, n_s) * v[n_s] for n_s in mdp.states])
                            for a in mdp.actions)
                                
        pol_π[s] = findmax(actions_value)[2]
    end
    
    return pol_π
end

iter_value (generic function with 1 method)

## Definiendo el proceso de decisión de Markov

En las siguientes celdas viene el código necesario para definir un proceso de decisión de Markov que represente el juego y que sea compatible con ``iter_value``.

Primero definiremos los estados del juego como una 2-tupla de tuplas (de hasta 4 elementos cada una) de números enteros.

In [3]:
states = [[2, i] for i in 2:26]

for i in 3:39
    push!(states, [3, i])
end

for i in 4:52
    push!(states, [4, i])
end

Ahora las acciones. Esta parte es fácil, solo hay dos posibles acciones en todo momento. Esta libreta usa el vocabulario usado en los casinos.

In [4]:
actions = ["hit", "stand"]

2-element Array{String,1}:
 "hit"  
 "stand"

Seguimos con la declaración de la función que calcule la probabilidad de transición entre estados.

In [5]:
function ρ(s, a, n_s)
    if a == "stand"
        if s == n_s
            return 1
        else
            return 0
        end
    else
        diff_score = n_s[2] - s[2]
        
        if n_s[1] == s[1] + 1 && diff_score >= 1 && diff_score <= 13 
            return 1/13
        end
    end
    
    return 0
end

ρ (generic function with 1 method)

Casi terminando, calculamos la recompensa para cada estado. Aquí no hay ganancia ni pérdida a menos que el juego se acabe.

In [6]:
function reward(s, a, n_s)
    if (n_s[1] == 4 && n_s[2] <= 21) || n_s[2] == 21
        return 1
    elseif n_s[2] > 21
        return -1
    else
        return 0
    end
end

reward (generic function with 1 method)

## Resolviendo el problema

Como tenemos todos los preparativos listos, podemos crear un nuevo proceso de decisión de Markov que modele este 21.

In [7]:
twenty_one = MDP(states, actions, ρ, reward, [])

MDP(Array{Int64,1}[[2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [2, 11]  …  [4, 43], [4, 44], [4, 45], [4, 46], [4, 47], [4, 48], [4, 49], [4, 50], [4, 51], [4, 52]], ["hit", "stand"], ρ, reward, Any[])

Y con esto, ahora podemos probar si nuestro algoritmo realmente cumple su tarea.

In [11]:
γ = 0.8
pol_π = iter_value(twenty_one, γ)

Dict{Array{Int64,1},String} with 111 entries:
  [3, 3]  => "stand"
  [4, 4]  => "stand"
  [4, 16] => "stand"
  [3, 25] => "stand"
  [3, 31] => "stand"
  [3, 29] => "stand"
  [3, 4]  => "hit"
  [4, 8]  => "stand"
  [4, 43] => "stand"
  [4, 52] => "stand"
  [2, 2]  => "hit"
  [4, 19] => "stand"
  [4, 51] => "stand"
  [3, 17] => "stand"
  [4, 48] => "stand"
  [3, 27] => "stand"
  [2, 14] => "stand"
  [4, 22] => "stand"
  [4, 36] => "stand"
  [2, 17] => "stand"
  [3, 33] => "stand"
  [4, 27] => "stand"
  [2, 24] => "stand"
  [3, 26] => "stand"
  [4, 38] => "stand"
  ⋮       => ⋮

Veamos los resultados. No muy buenos por ahora.

In [12]:
for s in states
    println(s, pol_π[s])
end

[2, 2]hit
[2, 3]stand
[2, 4]stand
[2, 5]stand
[2, 6]stand
[2, 7]stand
[2, 8]stand
[2, 9]stand
[2, 10]stand
[2, 11]stand
[2, 12]stand
[2, 13]stand
[2, 14]stand
[2, 15]stand
[2, 16]stand
[2, 17]stand
[2, 18]stand
[2, 19]stand
[2, 20]stand
[2, 21]stand
[2, 22]stand
[2, 23]stand
[2, 24]stand
[2, 25]stand
[2, 26]stand
[3, 3]stand
[3, 4]hit
[3, 5]hit
[3, 6]hit
[3, 7]hit
[3, 8]hit
[3, 9]hit
[3, 10]hit
[3, 11]hit
[3, 12]hit
[3, 13]hit
[3, 14]hit
[3, 15]hit
[3, 16]stand
[3, 17]stand
[3, 18]stand
[3, 19]stand
[3, 20]stand
[3, 21]stand
[3, 22]stand
[3, 23]stand
[3, 24]stand
[3, 25]stand
[3, 26]stand
[3, 27]stand
[3, 28]stand
[3, 29]stand
[3, 30]stand
[3, 31]stand
[3, 32]stand
[3, 33]stand
[3, 34]stand
[3, 35]stand
[3, 36]stand
[3, 37]stand
[3, 38]stand
[3, 39]stand
[4, 4]stand
[4, 5]stand
[4, 6]stand
[4, 7]stand
[4, 8]stand
[4, 9]stand
[4, 10]stand
[4, 11]stand
[4, 12]stand
[4, 13]stand
[4, 14]stand
[4, 15]stand
[4, 16]stand
[4, 17]stand
[4, 18]stand
[4, 19]stand
[4, 20]stand
[4, 21]stand
[4, 22]